## Loading data

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')


--2024-11-06 21:10:41--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.207, 142.250.31.207, 142.251.111.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   172MB/s    in 0.4s    

2024-11-06 21:10:42 (172 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Reading the Data into arrays

In [11]:
from PIL import Image
import numpy as np

cats_dir = os.path.join(train_dir, "cats")
dogs_dir = os.path.join(train_dir, "dogs")

# Initialize as lists
x_train = []
y_train = []
NUMBER_OF_EXAMPLES = 1000

i = 0
while i < NUMBER_OF_EXAMPLES:
    if i % 2 == 0:
        im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i])).convert("RGB")
        im_resized = im.resize((150, 150))
        x_train.append(np.array(im_resized))
        y_train.append(1)
    else:
        im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i])).convert("RGB")
        im_resized = im.resize((150, 150))
        x_train.append(np.array(im_resized))
        y_train.append(0)
    i += 1

# Convert lists to NumPy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)



## Beginning to define the model (this is where you come in, I loaded the pretrained model for you)

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    weights='imagenet'
)

# Freeze the convolutional base
pretrained_model.trainable = False

# Add custom layers on top
model = models.Sequential()
model.add(pretrained_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))



94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [14]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
              metrics=['accuracy'])

history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.9759 - loss: 0.0749 - val_accuracy: 0.9800 - val_loss: 0.0811
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 0.9713 - loss: 0.0859 - val_accuracy: 0.9800 - val_loss: 0.0786
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.9818 - loss: 0.0654 - val_accuracy: 0.9600 - val_loss: 0.0878
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.9808 - loss: 0.0684 - val_accuracy: 0.9800 - val_loss: 0.0771
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 87s 3s/step - accuracy: 0.9897 - loss: 0.0561 - val_accuracy: 0.9750 - val_loss: 0.0794


In [15]:
loss, accuracy = model.evaluate(x_train, y_train)
print(f'Validation accuracy: {accuracy}')


32/32 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - accuracy: 0.9904 - loss: 0.0509
Validation accuracy: 0.9869999885559082


In [18]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load an image for prediction
img_path = 'cat.jpg'  # Update this path to the actual image file
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# Make a prediction
prediction = model.predict(img_array)
print('Cat' if prediction < 0.5 else 'Dog')


1/1 ━━━━━━━━━━━━━━━━━━━━ 52s 52s/step
Dog
